In [1]:
import main

# Hack to fix encoding mistakes throughout workbook
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf-8')


[nltk_data] Downloading package stopwords to /home/justin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/justin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
%%time
base_path = main.get_path('linux')

# ----------------------------------------------------------------------------------
# Loads bibliometric data sheets

# Loads inspect data sheet
print('********************************************')
print('Loading inspec')
print('********************************************')
inspec_dict_list = main.load_inspec(base_path)
main.set_database('inspec', inspec_dict_list)

# Loads IEEE data sheet
print('********************************************') 
print('Loading IEEE explore')
print('********************************************')
ieee_dict_list = main.load_ieee_explore(base_path)
main.set_database('ieee', ieee_dict_list)

# Loads ACM new data sheet
print('********************************************')
print('Loading ACM new')
print('********************************************')
acm_new_dict_list = main.load_acm_new(base_path)
main.set_database('acm', acm_new_dict_list)

   # ----------------------------------------------------------------------------------
# Loads altmetric data sheets
print('********************************************')
print('Loading main altmetric')
print('********************************************')
altmetric_dict_list = main.load_main_altmetric(base_path)
main.set_database('alt', altmetric_dict_list)
# ----------------------------------------------------------------------------------

print('Appending dictionary lists')
dict_list =  inspec_dict_list + ieee_dict_list + acm_new_dict_list + altmetric_dict_list
# dict_list = acm_new_dict_list + ieee_dict_list 
print('Complete')

# Puts titles in a standard format
for line in dict_list:
    line['title'] = main.capitalize_title(line['title'])

# Check for exact title duplicates
# print('Searching for exact matches in the list')
# dict_list, exact_matches = compare.mark_exact_duplicates(dict_list, 'title')
# print('Complete')

# Text comparison 
print('Marking possible duplicates: ')
dict_list = main.compare.mark_possible_duplicates(dict_list, 'title')



In [3]:
import os
import xlwt
import csv
filename = os.path.join('/home/justin/Dropbox', 'test_output_no_global_encoding.csv')

In [12]:
# def assign_headers(key_list, data):
#     headers = [k for k, v in data[0].items()]
#     lower_key_list = [x.lower() for x in key_list]
#     print(lower_key_list)
    
#     for item in headers:
#         if item not in lower_key_list:
#             print(item)
            
#     print(lower_key_list)
#     for item in lower_key_list:
#         if item not in headers:
#             print(item)


def add_affiliation_details(institution, author_affiliations_list, countries_list):
    
    # Add all author_affiliations if not already in the list
    if not institution in author_affiliations_list:
        author_affiliations_list.append(institution)        
         
    affiliation_index = None
    for i, affiliation in enumerate(author_affiliations_list):
        if affiliation == institution:
            affiliation_index = i 
            
    return affiliation_index

def update_author_affiliations(author_name, authors_str, affiliation_indices):
    # Adds the number associated to the affiliations
    if affiliation_indices: 
        affiliation_index_str = '('
        for i, index in enumerate(affiliation_indices):
            affiliation_index_str += str(index+1)
            if i != len(affiliation_indices) - 1:
                affiliation_index_str += ','
        affiliation_index_str += ') ' 
    else:
        affiliation_index_str = ''
        
    return authors_str + author_name + ' ' + affiliation_index_str
    
def prepare_author_affiliations_str(author_affiliations_list):
    # Converts list to a numbered string
    author_affiliation_str = ''
    for i, affiliation in enumerate(author_affiliations_list):
        author_affiliation_str += '(' + str(i+1) + ')' + affiliation + ' '
    return author_affiliation_str.strip()

def format_author_details(author_keys, author_details):
    authors_str = ''
    author_affiliations_list = []
    author_affiliations_counter = 0
    countries_list = []
    departments_list = []
    country = ''
    
     # Check whether the instance is a list
    if isinstance(author_details, (list,)):             
        for author in author_details:
            # Add author index of affiliation        
            
            new_affiliation = True
            institutions = author['institutional affiliation']
            author_name = author['authors']
            if author['country']:
                country = author['country']
            
            affiliation_indices = []
            if isinstance(institutions, (list,)):
                for institution in institutions:
                    affiliation_indices.append(add_affiliation_details(institution, author_affiliations_list, countries_list))
            
            elif isinstance(institutions, basestring):
                affiliation_indices.append(add_affiliation_details(institutions, author_affiliations_list, countries_list))
            else:
                #TODO handle this properly
                pass
#                 print(type(institutions))
#                 print('Invalid institutional affiliation')
            
            authors_str = update_author_affiliations(author_name, authors_str, affiliation_indices)
            
    else:
        # Perform the same operations with single dictionary item
        print('Warning, author-details are not a list!')
        pass
    
    return authors_str, prepare_author_affiliations_str(author_affiliations_list), country
    
    
    
def export_dict_list_to_csv(headers, data, filename):
    try:
        os.remove(filename)
    except OSError:
        pass
    with open(filename, 'wb') as f:
        
        # Adds the headers to the csv
        csv_data = [headers]
        
        author_keys = ['authors', 'department', 'institutional affiliation', 'country']
        
        error_counter = 0
        for line in data:
            # Change 'None' assignments to empty strings for utf encoding step
            line = {h.lower(): (u'0' if line[h.lower()] is None else line[h.lower()]) for h in headers}
            row = []    
            institution_type_set = set()
            
            authors = ''
            institution = ''
            country = ''
            
            authors_str = 'authors'
            institution_str = 'institution'
            country_str = 'country'
            
            for h in headers:
                if h.lower() == authors_str:
                    # TODO add country
                    authors, institution, country = format_author_details(author_keys, line[authors_str])
                    row.append(authors.encode("utf-8"))
                elif h.lower() == institution_str:
                    row.append(institution.encode("utf-8"))
                elif h.lower() == country_str:
                    row.append(country.encode("utf-8"))
                else: 
                    row.append(str(line[h.lower()]).encode("utf-8"))
#                 except Exception as e:
#                     print(e)
#                     error_counter += 1
#                     row.append(u' '.join('Encoding error'.encode("utf-8")))
            
            csv_data.append(row) 
        print(institution_type_set)
            
        print('Errors: ' + str(error_counter))
        writer = csv.writer(f)
        writer.writerows(csv_data)

# assign_headers(main.final_key_list, dict_list)
export_dict_list_to_csv(main.final_key_list, dict_list, filename)

In [ ]:
dict_list = inspec_dict_list
for line in dict_list:
    
# Code to handle author disctionaries
columns = list(dict_list[0].keys())
with open(filename, "w") as f:
    w = csv.DictWriter(f, header)
    w.writeheader()
    for key, val in sorted(d.items()):
        row = {'Name': key}
        row.update(val)
        row['Car'] = '; '.join(row['Car'])  # here
        row['House'] = '; '.join(row['House']) # here
        w.writerow(row)

In [ ]:
def output_to_csv(dict_list):
    
    filename = os.path.join(os.getcwd(), 'output.xls')
    try: 
        os.remove(filename)
    except:
        pass

    workbook = xlwt.Workbook()
    ws = workbook.add_sheet('test')

    columns = list(dict_list[0].keys())

    # write headers in row 0
    for j, col in enumerate(columns):
        ws.write(0, j, col)

    for i, row in enumerate(dict_list, 1):
        for j, col in enumerate(columns):
            ws.write(i, j, row[col])

    workbook.save(filename)

In [ ]:
# TODO sort by similarity cluster
# TODO output author on individual line along with author data such as country
# TODO may need unicode conversion for row objects
    # self.writer.writerow([unicode(s).encode("utf-8") for s in row])